<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/Llama3_2_1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

dataset = load_dataset("Owen313/chinese-novel-dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset.json:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/729 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# load
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [5]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377
None


q_proj and v_proj are linear projection layers, which are responsible for mapping the input embedding into the representation of Query and Value:

*Q_proj:* Generate a Query from the input to calculate the attention weight.

*V_proj:* Generates Value from the input, which is used to calculate the final output by weighted sum.

Query and Value are the key parts of attention mechanism, which have great influence on model performance and presentation ability.
Only adjusting the parameters of these two parts can significantly affect the behavior of the model and avoid adjusting the weight of the whole model.

In [ ]:
def preprocess_function(examples):
    inputs = [ex for ex in examples["input"]]
    outputs = [f"<|output|> {out}" for out in examples["output"]]
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    model_inputs = tokenizer(inputs, padding="max_length", max_length=128, truncation=True)
    labels = tokenizer(outputs, padding="max_length", max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

print(tokenized_datasets)

Map:   0%|          | 0/729 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 729
    })
})


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    # eval_strategy="epoch",
    # eval_dataset='Trainer',
    run_name="DMGY_3",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"].train_test_split(test_size=0.2),
)

trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,6.981400
1000,6.647600
1500,6.609200
2000,6.559600
2500,6.501700
3000,6.454300
3500,6.407600


TrainOutput(global_step=3640, training_loss=6.587406820779318, metrics={'train_runtime': 2559.5438, 'train_samples_per_second': 2.848, 'train_steps_per_second': 1.422, 'total_flos': 5450425612369920.0, 'train_loss': 6.587406820779318, 'epoch': 9.986282578875171})

In [10]:
test_inputs = [
    "我是",
    "他们",
    "Here we are",
    "我是Chinese"
]
model.to("cuda")

for test_input in test_inputs:
    inputs = tokenizer(test_input, return_tensors="pt")
    inputs = {key: value.to("cuda") for key, value in inputs.items()}
    output_ids = model.generate(**inputs, max_length=50)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"Input: {test_input}")
    print(f"Generated output: {output_text}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input: 我是
Generated output: 我是中国人，和你一样。
我和你一样，吃饭的时候有饭，喝水的时候有水。
我和你一样，穿衣服的时候有衣服，穿鞋的时候有鞋。
我和你一样，睡



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input: 他们
Generated output: 他们都很年轻，穿着很现代。 The men are young, they are dressed very modern.
A. 正确
B. 错误
Answer: A



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input: Here we are
Generated output: Here we are at the end of the month, and I'm sure I'm not the only one who is still working on the goals I set for the month. I've been so busy with work and my personal life that I haven't had

Input: 我是Chinese
Generated output: 我是Chinese. 我是中国人.
I'm Chinese. I'm Chinese.
I'm Chinese. I'm Chinese.
I'm Chinese. I'm Chinese.
I'm Chinese. I'm Chinese.
I'm Chinese. I'm Chinese.


